# Building an Assembly $K_\infty$ Model

## Goals and Steps
1. Take previous pin-cell model
2. Implement Universes, and lattices
    1. Create universe for pin cell
    2. Create lattice cell
4. Create a fuel pin only assembly

Based on BEAVRS:
* N. E. Horelik et al., "Benchmark for Evaluation and Validation of Reactor Simulations (BEAVRS)," presented at the Int. Conf. Mathematics and Computational Methods Applied to Nuc. Sci. & Eng., Sun Valley, Idaho, 2013.


# Step 1: Starting
1. Import modules

In [ ]:
%pip install montepy
import montepy
import numpy as np
import warnings

warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow")
# In this case we do not care about lines expanding
warnings.simplefilter("ignore", montepy.errors.LineExpansionWarning)

2. Load previous problem
  * Valid model at `models/pin_cell_corrected_ans.imcnp`
  * Or, use your own model from Notebook 1!

In [ ]:
problem = montepy.read_input("models/pin_cell_corrected_ans.imcnp")

# Background
* Modeling Westinghouse 17$\times$17 assembly
* Has square lattice of pin cells (17$\times$17)

In [ ]:
# Overall assembly pitch/side length
ASSEMBLY_PITCH = 21.50  # [cm]
# size of pin cell
PIN_PITCH = 1.26  # [cm]
# Number of pins per size
NUM_PINS_SIDE = 17  # [-]

# Step 2.1: Create Pin cell universe
* Create a new [Universe](https://www.montepy.org/en/stable/api/montepy.universe.html),
* add the universe to [problem.universes](https://www.montepy.org/en/stable/api/montepy.mcnp_problem.html#montepy.mcnp_problem.MCNP_Problem.universes),
* add all cells to that [Universe](https://www.montepy.org/en/stable/api/montepy.universe.html).

In [ ]:
# Task: Create a new universe
universe = None
if universe is not None:
    problem.universes.append(universe)
    universe.claim(problem.cells)
    universe

# Step 2.2: Remove Reflective Boundaries
* Iterate over all surfaces
* Update the `is_reflecting` property
* Make sure to exclude z-planes (`PZ`).

In [ ]:
# Task: Make it radially infinite (keep it axially finite)
for surf in problem.surfaces:
    print(surf.surface_type)
    pass

# Step 2.3: Define unit cell of lattice
* Remember that order matters for surface definition when using `LAT`
   * Parrallel surfaces must be next to each other in defintion order, and pairs define lattice coordinate system
   * will use the following order: `x`, `y`, `z`
* Unit cell defines lattice site: `[0,0,0]`
* Will define assembly origin to be center of this unit cell
* For convenience the numbers of the surfaces that will be used are given:

In [ ]:
# We will cheat a little and give you the surfaces by number
# (Extra credit: find these automatically from the surface type and coefficient)
surfs = problem.surfaces
right_surf = surfs[104]
left_surf = surfs[103]
y_top_surf = surfs[106]
y_bot_surf = surfs[105]
z_top_surf = surfs[102]
z_bot_surf = surfs[101]

# Defining geometry

* Geometry is defined by bitwise operators
   * `&` for intersection
   * `|` for union
   * `~` for complement
* Half-spaces (from surfaces) are defined by `+`, `-`
* [Guide for more information](https://www.montepy.org/en/stable/starting.html#geometry)


## For Example

* Start with a z-plane:


## These are demos for the instructor to do and students to follow

In [ ]:
print(z_bot_surf)

* Get the top/above half-space

In [ ]:
print(+z_bot_surf)
print(type(+z_bot_surf))

* Make the union of the two half-spaces

In [ ]:
print(+z_bot_surf | -z_bot_surf)

# Goal:
1. Create a unit cell
2. assign it a number and append to the cells
    * see: [`Cell.number`](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.number), and [`Cells.request_number`](https://www.montepy.org/en/stable/api/montepy.cells.html#montepy.cells.Cells.request_number)
3. Define the unit cell [`geometry`](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.geometry) using the previous surfaces
    * below the right surface, and above the left surface, and...
5. Set the neutron [importance](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.importance) to 1.0


**Note:** In MCNP, unit cells are generally void: we will not need a material or density.

In [ ]:
# make the cell
unit_cell = montepy.Cell()
# request a new number and append
unit_cell.number = problem.cells.request_number()
# define geometry
unit_cell.geometry = -right_surf & +left_surf
unit_cell.geometry &= -y_top_surf & +y_bot_surf
unit_cell.geometry &= -z_top_surf & +z_bot_surf
# set importance
unit_cell.importance.neutron = 1.0
print(unit_cell)
print(unit_cell.mcnp_str())

In [ ]:
problem.cells.append(unit_cell)

# Set the Unit Cell Lattice and Fill information

## Defining a lattice unit Cell in MCNP

1. Need a void (there are exemptions) cell to be the unit cell
2. That unit cell needs to be `fill`ed with the universe that will make up that unit cell
3. The unit cell has to have a `lat`tice type defined
4. The unit cell needs to be placed in its own universe
     * This isn't always strictly necessary but allows you to avoid making infinite lattices

# Tools you will need from MontePy
* Will use the [`Cell.lattice_type`](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.lattice_type), [`cell.universe`](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.universe), and [`Cell.fill`](https://www.montepy.org/en/stable/api/montepy.data_inputs.fill.html#montepy.data_inputs.fill.Fill) properties
    * `Cell.lattice` requires a `LatticeType` enum instance which is accessible as `montepy.LatticeType`
    * `Cell.fill` is a bitt more complicated because there are multiple options for `Fill`
    * For now we will only be using [`Cell.fill.universe`](https://www.montepy.org/en/stable/api/montepy.data_inputs.fill.html#montepy.data_inputs.fill.Fill.universe)
* Need to assign to another universe, which will fill the assembly

In [ ]:
unit_cell.lattice_type = montepy.LatticeType.HEXAHEDRAL
unit_cell.fill.universe = universe

# Task: assign to own universe
lat_universe = None

# Task: Either set the unit_cell's universe to 'lat_universe', or claim the unit cell from the lat_universe
lat_universe

In [ ]:
problem.universes.append(lat_universe)

# Step 3.1 Create bounding box of lattice
* Need to constrain lattice to not be infinite
* Use [clone](https://www.montepy.org/en/stable/api/montepy.surfaces.surface.html#montepy.surfaces.surface.Surface.clone) for x and y planes
   
 


## Goals
1. Make a new bounding cell
1. assign number, and append
1. define geometry
    1. clone surfaces
    1. Update side locations (keep bottom left corner the same though)
    1. define geometry
1. Fill with lattice universe

In [ ]:
# Create the new cell
lattice_bound = montepy.Cell()
lattice_bound.number = problem.cells.request_number()

In [ ]:
# clone surfaces
outer_right_surf = right_surf.clone()
outer_left_surf = left_surf.clone()
outer_y_top = y_top_surf.clone()
outer_y_bot = y_bot_surf.clone()

# Task: shift locations
edge_shift = 0.0  # Calculate where the LATTICE edge needs to be now, in cm
outer_right_surf.location = -PIN_PITCH / 2.0 + edge_shift
outer_y_top.location = -PIN_PITCH / 2.0 + edge_shift

# Task: define the LATTICE region from the 4 outer surfaces
lattice_bound.geometry = None

# fill
lattice_bound.fill.universe = lat_universe

In [ ]:
problem.cells.append(lattice_bound)

# Step 3.1.2: Avoid coincident surfaces

* Right now the lattice bounding cell, and the water in the cell use the same surfaces.
* This can lead to geometry errors.
* We will just update the unit cell's water to be an infinite region, which gets truncated.



## Steps to accomplish this:
1. Grab the largest cylinder `CZ` surface
2. Grab the water cell
3. Set the water cell to only be outside said cylinder

In [ ]:
# provided
cyl_sorter = lambda surf: surf.radius
water_cyl = max(problem.surfaces.cz, key=cyl_sorter)

# grab water cell
water_mat = list(problem.materials.get_containing_all("H", "O"))[0]
water_cell = list(water_mat.cells)[0]
water_cell.geometry = +water_cyl
print(water_cell.mcnp_str())

# Step 3.2: Finish the assembly
* Now we have a 17$\times$17 grid of fuel pins, however this isn't an assembly yet.
   1. extra water around edge of assembly
   2. Need reflective boundaries for $k_\infty$

# Find Perimeter width
1. Calculate amount of extra width around the pins
2. Divide by two to get width of perimeter
3. [Round](https://numpy.org/doc/stable//reference/generated/numpy.round.html) to 5 decimal places

In [ ]:
extra_width = ASSEMBLY_PITCH - NUM_PINS_SIDE * PIN_PITCH
perimeter = np.round(extra_width / 2, 5)
perimeter

# Make Cell and new Surfaces
1. Clone previous x, y surfaces
2. Shift surfaces out (`+=` and `-=` are your friends)
3. Set surfaces as reflecting
4. make new cell (remember to carve out the `lattice_bound` cell)

In [ ]:
# Create the outer perimeter water cell
perimeter_water = montepy.Cell()
perimeter_water.number = problem.cells.request_number()
perimeter_water.material = water_mat
perimeter_water.mass_density = 1.0

edge_right = outer_right_surf.clone()
edge_left = outer_left_surf.clone()
edge_y_top = outer_y_top.clone()
edge_y_bot = outer_y_bot.clone()

# shift in +y, +x
for surf in [edge_right, edge_y_top]:
    surf.location += perimeter
    surf.is_reflecting = True

# shift in -y, -x
for surf in [edge_left, edge_y_bot]:
    surf.location -= perimeter
    surf.is_reflecting = True

In [ ]:
# Now set the geometry of the outer water perimeter
perimeter_water.geometry = +edge_left & -edge_right
perimeter_water.geometry &= -edge_y_top & +edge_y_bot
perimeter_water.geometry &= +z_bot_surf & -z_top_surf
# Task: Put the perimeter water outside the lattice edges
perimeter_water.geometry &= None
print(perimeter_water.mcnp_str())

In [ ]:
problem.cells.append(perimeter_water)

# Fixing Cell Importances

* Have added multiple new cells with default importance of 0.0 
* Would lead to problems.
* Want to set importance to 1.0 everywhere.
* Use [`MCNP_Problem.cells.set_equal_importance()`](https://www.montepy.org/en/stable/api/montepy.cells.html#montepy.cells.Cells.set_equal_importance)

In [ ]:
problem.cells.set_equal_importance(1.0)
print([cell.importance.neutron for cell in problem.cells])

# Write it out to file

In [ ]:
problem.write_problem("models/oops_all_pins.imcnp", overwrite=True)

# Results: Plot

## Questions?
![plot of a westinghouse 17 by 17 fuel assembly but with only fuel assemblies.](figs/all_pins.png)